## Sliding Tile Puzzle

In [42]:
from collections import deque
from copy import deepcopy


"""
Helper Functions
"""

def index_2d(array_2d: list[list], target: int):
    for i, row in enumerate(array_2d):
        for j, val in enumerate(row):
            if val == target:
                return (i, j)
    return None

def tuple_2d(array_2D):
    return tuple(map(tuple, array_2D))

In [43]:
def count_inversion(state):
    flat = [item for row in state for item in row if item != 0]

    return sum(flat[i] > flat[j] for i in range(len(flat)) for j in range(i + 1, len(flat)))


def is_solvable(state):
    """
    Determine if a state is solvable.
    
    Parameters:
        state (2D_array): A state of the puzzle.

    Returns:
        bool: Return `True` if solvable, `False` otherwise.
    """
    length, width = len(state), len(state[0])

    # Count inversion
    inversions = count_inversion(state)
    
    # Parity check
    if width % 2 != 0:  # if width is odd
        return inversions % 2 == 0
    if (index_2d(state, 0)[0] % 2 == 0) == (length % 2 == 0):  # If black tile is on even row
        return inversions % 2 != 0
    return inversions % 2 == 0

In [44]:
states = [[[5, 4, 3],
           [2, 1, 0]],
          [[7, 6, 5, 0],
           [4, 3, 2, 1]],
          [[7, 6, 5, 4],
           [3, 2, 1, 0]]]
for state in states:
    if is_solvable(state):
        print(f"solvable\n{state}")
    else:
        print(f"unsolvable\n{state}")

solvable
[[5, 4, 3], [2, 1, 0]]
solvable
[[7, 6, 5, 0], [4, 3, 2, 1]]
unsolvable
[[7, 6, 5, 4], [3, 2, 1, 0]]


In [45]:
a = [[2, 5], [6, 1]]
sorted(a)

[[2, 5], [6, 1]]

In [51]:
def get_child(parent_state):
    """
    Determine every child states can be reached from the parent state.
    
    From the parent state, performing every possible 1-sliding-move on the blank tile. Some\
    possible moves are sliding up (U), sliding down (D), sliding (L), and sliding right (R).

    Parameters:
        parent_state (2D_array): The parent state.

    Returns:
        out (1D_array): List of child states can be reached. Return empty list if there is no child state.    
    """
    def child(move):
        """
        Perform 1 move from the current state and return the destination state.
        """
        state = deepcopy(parent_state)
        cur_i, cur_j = index_2d(parent_state, 0)
        des_i, des_j = cur_i, cur_j
        
        if move == 'U':
            des_i -= 1
        elif move == 'D':
            des_i += 1
        elif move == 'L':
            des_j -= 1
        elif move == 'R':
            des_j += 1
        
        state[cur_i][cur_j], state[des_i][des_j] = state[des_i][des_j], state[cur_i][cur_j]
        return state

    
    child_states = []

    length, width = len(parent_state), len(parent_state[0])
    blank_id = index_2d(parent_state, 0)
    if blank_id[0] - 1 >= 0:  # Slide up
        child_states.append(child('U'))
    if blank_id[0] + 1 < length:  # Slide down
        child_states.append(child('D'))
    if blank_id[1] - 1 >= blank_id[1] // width * width:  # Slide left
        child_states.append(child('L'))
    if blank_id[1] + 1 < blank_id[1] // width * width + width:  # Slide right
        child_states.append(child('R'))

    return child_states


def check_goal(state_2d):
    """
    Determine if input state is a goal state. Goal state is pre-determined by the problem.
    """
    length, width = len(state), len(state[0])
    
    goal_state_1d = list(range(1, length * width)) + [0]
    for i, row in enumerate(state_2d):
        for j, val in enumerate(row):
            if goal_state_1d[i * width + j] != state_2d[i][j]:
                return False
    return True


def solution(goal_state, parent):
    """
    Return backtracking path from goal state to initial state.

    Parameters:
        goal_state (2D_array): The goal state.
        parent (dict{ tuple: list }): A dictionary that stores reference to parent state of a child state. Reference\
            to parent state of the initial state is `None`.
    
    Returns:
        out (1D_array): The backtracking path from goal state to initial state.
    """
    backtrack_path = [goal_state]    
    temp_state = goal_state
    while parent[tuple_2d(temp_state)]:
        temp_state = parent[tuple_2d(temp_state)]
        backtrack_path.append(temp_state)
    return backtrack_path


def bfs(initial_state):
    """
    Solving Sliding Tile Puzzle with BFS.
    
    Parameters:
        initial_state (2D_array): A state of the puzzle from where BFS start traverse.
    
    Returns:
        solution (1D_array, or None): If goal state is reached, return a backtracking path\
            . Return `None` otherwise.
    """
    frontier = deque([initial_state])
    parent = {tuple_2d(initial_state): None}

    # Check solvability
    if not is_solvable(initial_state):
        return None

    # If initial state is goal state
    if check_goal(initial_state):
        return solution(initial_state, parent)
    
    # Traverse tree
    goal_state = None
    while (not goal_state) and (frontier):
        parent_state = frontier.popleft()
        child_states = get_child(parent_state)
        
        for child_state in child_states:
            if (tuple_2d(child_state) not in parent):  # If not explored
                parent[tuple_2d(child_state)] = parent_state  # Mark explored, and add parent state for backtracking
                if check_goal(child_state):
                    goal_state = child_state
                    break
                frontier.append(child_state)  # Add queue for traversing

    if goal_state:
        return solution(goal_state, parent)
    return None

In [53]:
initial_state = [[2, 0],
                 [1, 3]]
path = bfs(initial_state)
if path:
    print("Backtracking from goal state...")
    for state in path:
        print(state)
        print()
else:
    print("No goal state reached!")

Backtracking from goal state...
[[1, 2], [3, 0]]

[[1, 2], [0, 3]]

[[0, 2], [1, 3]]

[[2, 0], [1, 3]]



In [52]:
state = [[1, 2],
         [3, 0]]

check_goal(state)

True